In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 363.4/363.4 MB 107.8 MB/s eta 0:00:01

In [ ]:
# 🧠 Step 1: Load Qwen3-4B from Hugging Face load locally

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score


In [ ]:
#Step2. load model
model_id = "Qwen/Qwen3-4B"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id,
    # load_in_8bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

print("✅ Model and tokenizer loaded successfully.")



In [ ]:
# 🧾 Step 3: Define the prompt template

def build_prompt(input_finding):
#    return f"""
#Your task is to extract disease and abnormal findings from the following radiology report.
    return f"""You are a clinical AI assistant. Extract only the confirmed or highly suspected disease names from this radiology report.

Instructions:
- Do not include any explanation, reasoning, or additional text.
- Return only the disease names, separated by commas.
- Use standard medical terminology.
- Be precise and avoid speculation.
- Do not duplicate disease names.
- If no diseases are found, output: No significant findings.

Now analyze this radiology report:
{input_finding}

output_disease\n"""



In [ ]:
# 🧼 Step 4: Clean and parse output

def parse_labels(output_string, delimiter="output_disease"):
    """
    Extracts a list of disease/abnormality labels from the model's raw output.

    - Splits on the given delimiter (e.g. "output_disease")
    - Detects whether the body is comma-separated or bullet-list
    - Cleans up whitespace and leading bullets/hyphens
    - PRESERVES all entries (no deduplication)
    """
    # 1) Isolate the part after your delimiter
    try:
        body = output_string.split(delimiter, 1)[1]
    except IndexError:
        body = output_string

    # 2) Trim whitespace
    body = body.strip()

    labels = []
    # 3a) If there's a comma on the first line, treat as comma-separated
    first_line = body.splitlines()[0]
    if "," in first_line:
        parts = [p.strip() for p in first_line.split(",")]
        labels = [p for p in parts if p]

    # 3b) Otherwise, one-item-per-line list
    else:
        for line in body.splitlines():
            clean = line.strip()
            if not clean:
                continue
            # remove leading bullets or hyphens
            clean = re.sub(r"^[\-\u2022]\s*", "", clean)
            # split if there are commas in a bullet
            for part in clean.split(","):
                part = part.strip()
                if part:
                    labels.append(part)


In [ ]:
# 🧪 Step 5: Example inference
example_input = """The liver is normal in size and shape with homogeneous density. A patchy low-density lesion is seen around the liver fissure. The intrahepatic duct system is not obviously dilated, and the course is normal. The gallbladder is not enlarged, with no obvious thickening of the wall, and no clearly abnormal density foci are seen inside. The spleen is normal in size and shape with homogeneous density, and some punctate low-density foci are seen inside. The pancreas is normal in size and shape with homogeneous density, and no clearly abnormal density foci are seen inside. The main pancreatic duct is not obviously dilated, and the peripancreatic fat space is clear. Both kidneys are normal in size and shape with homogeneous density. A round low-density lesion is seen in the right kidney with a diameter of about 16mm. The left adrenal gland is thickened, and a punctate high-density lesion is seen in the right adrenal gland. The renal pelvis-calyx system is not obviously dilated. The perirenal fat space is clear, and no clearly abnormal density foci are seen. No enlarged lymph nodes are seen in the retroperitoneum."""
expected_output = ["renal cyst", "adrenal hyperplasia", "adrenal calcification"]

prompt = build_prompt(example_input)
response = pipe(prompt, max_new_tokens=32, do_sample=False)[0]["generated_text"]
prediction = parse_labels(response, delimiter="output_disease")
print("🧠 Predicted:", prediction)

print("\n \n \n full responce: \n", response)


In [ ]:
# 🧮 Step 6: Metric calculation

def compute_metrics(preds, targets):
    # preds, targets: lists of label‐lists
    mlb = MultiLabelBinarizer()
    mlb.fit(preds + targets)  # <<< fit on all individual lists

    y_pred = mlb.transform(preds)
    y_true = mlb.transform(targets)

    return {
        "precision": precision_score(y_true, y_pred, average='micro', zero_division=0),
        "recall":    recall_score(y_true, y_pred, average='micro', zero_division=0),
        "f1_micro":  f1_score(y_true, y_pred, average='micro', zero_division=0),
        "jaccard":   jaccard_score(y_true, y_pred, average='samples', zero_division=0).item(),
    }

# Parse into a list
pred_labels = parse_labels(response, delimiter="output_disease")
pred_labels=[lbl.lower() for lbl in pred_labels]


# Prepare ground truth
true_labels = [lbl.strip().lower() for lbl in expected_output]

# Compute metrics
metrics = compute_metrics([pred_labels], [true_labels])
print("📊 Evaluation:", metrics)
